In [2]:
import torch
import os
import sys
sys.path.append('/data/zbw/course/AAAI/project/work/')
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

class MinistDataLoader(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.samples = []
        self.transform = transform

        for label in os.listdir(root_dir):
            label_path = os.path.join(root_dir, label)
            for file in os.listdir(label_path):
                self.samples.append((os.path.join(label_path, file), int(label)))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        file_path, label = self.samples[idx]
        sample = np.load(file_path)
        
        sample = torch.from_numpy(sample)

        active_channel = None
        for i, channel in enumerate(sample):
            if torch.count_nonzero(channel) > 0:
                active_channel = i
                break
        
        
        if self.transform:
            sample = self.transform(sample)

        return sample, active_channel, label

path = '/data/zbw/course/AAAI/project/work'
batch_size = 64
train_data = MinistDataLoader(root_dir= path  + '/processed_data/train/')
train_data_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=2)

val_data = MinistDataLoader(root_dir= path  +'/processed_data/val/')
val_data_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=2)

class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)  
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1   = nn.Linear(16*4*4, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = x.view(x.size(0), -1) 
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


model_list = []
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
for i in range(0, 10):
    model_list.append(LeNet5().to(device))

optimizer_list = []
for i in range(0, 10):
    optimizer_list.append(optim.SGD(model_list[i].parameters(), lr=0.001, momentum=0.9))

criterion = nn.CrossEntropyLoss()

num_epochs = 10

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_data_loader, 0):
        env = data[1]
        inputs, labels = data[0].to(device), data[2].to(device)

        optimizer_list[env].zero_grad()

        outputs = model_list[env](inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer_list[env].step()

        running_loss += loss.item()
        if i % 100 == 99:   
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

torch.Size([64, 10, 28, 28]) torch.Size([64]) torch.Size([64])
torch.Size([64, 10, 28, 28]) torch.Size([64]) torch.Size([64])
torch.Size([64, 10, 28, 28]) torch.Size([64]) torch.Size([64])
torch.Size([64, 10, 28, 28]) torch.Size([64]) torch.Size([64])
torch.Size([64, 10, 28, 28]) torch.Size([64]) torch.Size([64])
torch.Size([64, 10, 28, 28]) torch.Size([64]) torch.Size([64])
torch.Size([64, 10, 28, 28]) torch.Size([64]) torch.Size([64])
torch.Size([64, 10, 28, 28]) torch.Size([64]) torch.Size([64])
torch.Size([64, 10, 28, 28]) torch.Size([64]) torch.Size([64])
torch.Size([64, 10, 28, 28]) torch.Size([64]) torch.Size([64])
torch.Size([64, 10, 28, 28]) torch.Size([64]) torch.Size([64])
torch.Size([64, 10, 28, 28]) torch.Size([64]) torch.Size([64])
torch.Size([64, 10, 28, 28]) torch.Size([64]) torch.Size([64])
torch.Size([64, 10, 28, 28]) torch.Size([64]) torch.Size([64])
torch.Size([64, 10, 28, 28]) torch.Size([64]) torch.Size([64])
torch.Size([64, 10, 28, 28]) torch.Size([64]) torch.Siz